In [ ]:
import numpy as np
import flammkuchen as fl
from pathlib import Path
import json
import tifffile as tiff

In [ ]:
master =  Path(r"/mnt/b4b63ff6-d9bf-4a97-9404-ce61fe415540/Home-Appliances/zenodo/Lavian_et_al_2025/data")
fish_list = [
    f for f in master.rglob("*_f*")
    if (f / "original").exists()
    and any((f / "original").glob("*metadata.json"))
    and any((p / "data_from_suite2p_unfiltered.h5").exists() for p in (f / "suite2p").glob("00*"))
]
fish = fish_list[-1]
fish

In [ ]:
for fish in fish_list:
    if not(fish / "anatomy_from_suite2p.tif").exists():
        print(fish)
        #getting anatomy stack dimensions from original:    
        with open(next((fish / "original").glob("*metadata.json")), "r") as f:
             stack_shape = json.load(f)['shape_full']

        suite2p_list = fish / 'suite2p'
        plane_list = list(suite2p_list.glob('*00*'))
        anatomy = np.zeros((stack_shape[1], stack_shape[-2], stack_shape[-1]))

        i = 0
        for path in plane_list:
            anatomy[i] = fl.load(path / 'data_from_suite2p_unfiltered.h5')['anatomy_stack']
            i += 1

        tiff.imsave(str(fish / "anatomy_from_suite2p.tif"), anatomy)